In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import l2_regularizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from prettytable import PrettyTable
import cv2
TRAIN_PIC_PATH = '/Users/apple/Desktop/FatigueDetection/FACE_BAG/MultiPIE/train_image128x128/'
TRAIN_LAB_PATH = TRAIN_PIC_PATH + 'Label.txt'
TEST_PIC_PATH = '/Users/apple/Desktop/FatigueDetection/FACE_BAG/yalefaces_/'
TEST_LAB_PATH = TEST_PIC_PATH + 'Label.txt'
PIC_SIZE = 64
LEARNING_RATE = 1e-5
REGULARIZATION_RATE = 1e-3
EPOCHs = 50
INPUT_NODE = PIC_SIZE * PIC_SIZE
OUTPUT_NODE = 2
scaler = MinMaxScaler()

In [2]:
def StandardImage(img): 
    img = cv2.resize(img,(PIC_SIZE,PIC_SIZE))
    img = cv2.equalizeHist(img)
    img = cv2.GaussianBlur(img, (3, 3), 0, 0)
    return img

In [3]:
X_train = np.zeros((2019,PIC_SIZE*PIC_SIZE))
Y_train = np.zeros((2019,1))
with open(TRAIN_PIC_PATH + 'FatigueNum.txt','r') as file:
    FatigueNum = np.array(map(int,file.read().split()))
with open(TRAIN_PIC_PATH + 'AwakeNum.txt','r') as file:
    AwakeNum = np.array(map(int,file.read().split()))
cnt_all = 0
cnt_awake = 0
cnt_tired = 0
cnt_tmp = 0
print AwakeNum.shape[0]
for i in range(2020):
        cnt_tmp = 0
        if cnt_all == 2019:
            break
        for j in range(25):
                if cnt_all < 2019:
                    #print cnt_all
                    if (j %5 ==0 or j%5 ==1) and cnt_tired < FatigueNum.shape[0]:
                        img = cv2.imread(TRAIN_PIC_PATH + str(FatigueNum[cnt_tired])+'.jpg',0)
                        img = StandardImage(img)
                        Y_train[cnt_all] = 0
                        X_train[cnt_all] = img.flatten()
                        cnt_tired+=1
                        cnt_all+=1 
                        #print 'cnt_all %d cnt_tired %d' % (cnt_all,cnt_tired)
                    elif cnt_awake < AwakeNum.shape[0] and cnt_tmp < 15:
                        cnt_tmp += 1
                       # print cnt_tmp
                        img = cv2.imread(TRAIN_PIC_PATH + str(AwakeNum[cnt_awake])+'.jpg',0)
                        img = StandardImage(img)
                        Y_train[cnt_all] = 1
                        X_train[cnt_all] = img.flatten()
                        cnt_awake+= 1
                        cnt_all += 1
X_train = scaler.fit_transform(X_train)
Y_train = OneHotEncoder().fit_transform(Y_train).todense()
X_test = np.zeros((176,PIC_SIZE*PIC_SIZE))
for i in range(176):
    image = cv2.imread(TEST_PIC_PATH+str(i+1)+'.jpg',0)
    image = StandardImage(image)
    X_test[i] = image.flatten()
X_test = scaler.transform(X_test)
Y_test = np.loadtxt(TEST_LAB_PATH).astype('int').reshape((-1,1))
Y_test = OneHotEncoder().fit_transform(Y_test).todense()
print X_train.shape,X_test.shape

1229
(2019, 4096) (176, 4096)


/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [4]:
with tf.name_scope('Inputs'):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name = 'x_in')
    y = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name = 'y_in')
    keep_prob = tf.placeholder(tf.float32,name = 'keep_prob')
    x_image = tf.reshape(x,[-1,PIC_SIZE,PIC_SIZE,1],name = 'x_in_image')

In [5]:
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name = name)

In [6]:
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name = name)

In [7]:
def conv2d(x,W,name):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME',name = name)

In [8]:
def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize =[1,2,2,1], strides=[1,2,2,1] ,padding = 'SAME',name = name)

In [9]:
with tf.name_scope('Conv1'):
    W_conv1 = weight_variable([5,5,1,32],'W_conv1')
    b_conv1 = bias_variable([32],'b_conv1')
    conv1 = conv2d(x_image,W_conv1,'conv1')
    h_conv1 = tf.nn.relu(conv1 + b_conv1,name = 'Relu_1')
    h_pool1 = max_pool_2x2(h_conv1,'h_pool1')
    tf.summary.histogram('W_conv1',W_conv1)


In [10]:
with tf.name_scope('Conv2'):
    W_conv2 = weight_variable([3,3,32,64],'W_conv2')
    b_conv2 = bias_variable([64],'b_conv2')
    conv2 = conv2d(h_pool1,W_conv2,'conv2')
    h_conv2 = tf.nn.relu(conv2 + b_conv2,name = 'Relu_2')
    h_pool2 = max_pool_2x2(h_conv2,'h_pool2')
    tf.summary.histogram('W_conv2',W_conv2)
    

In [11]:
with tf.name_scope('Conv3'):
    W_conv3 = weight_variable([5,5,64,128],'W_conv3')
    b_conv3 = bias_variable([128],'b_conv3')
    conv3 = conv2d(h_pool2,W_conv3,'conv3')
    h_conv3 = tf.nn.relu(conv3 + b_conv3,name = 'Relu_3')
    h_pool3 = max_pool_2x2(h_conv3,'h_pool3')
    tf.summary.histogram('W_conv3',W_conv3)

In [12]:
with tf.name_scope('fc1'):
    W_fc1 = weight_variable([8*8*128,512],'W_fc1')
    b_fc1 = bias_variable([512],'b_fc1')
    h_pool3_flat = tf.reshape(h_pool3,[-1,8*8*128],name = 'h_pool3_flat')
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat,W_fc1) + b_fc1)
    tf.summary.histogram('W_fc1',W_fc1)

In [13]:
with tf.name_scope('DropOut'):
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name = 'h_fc1_drop')

In [14]:
with tf.name_scope('output'):
    W_fc2 = weight_variable([512,OUTPUT_NODE],'W_fc2')
    b_fc2 = bias_variable([OUTPUT_NODE],'b_fc2')
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2) + b_fc2,name = 'Relu_4')
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2,name = 'y_conv2')
    tf.summary.histogram('W_fc2',W_fc2)

In [15]:
with tf.name_scope('loss_function'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv,labels = y),name = 'cross_entropy')
    l2 = l2_regularizer(REGULARIZATION_RATE)
    loss = cross_entropy + l2(W_conv1) + l2(W_conv2) + l2(W_conv3) + l2(W_fc1) + l2(W_fc2)
    tf.summary.scalar('loss',loss)

In [16]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [17]:
with tf.name_scope('prediction'):
    correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',accuracy)

In [18]:
def get_batch(X,Y,step,batch_size):
    for i in range(step // batch_size + 1):
        batch_x = X[i*batch_size:min(i*batch_size + batch_size,step)]
        batch_y = Y[i*batch_size:min(i*batch_size + batch_size,step)]
        yield batch_x,batch_y,i

In [24]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs/',sess.graph)
    merged = tf.summary.merge_all()
    tf.global_variables_initializer().run()
    for epoch in range(10):
        for bx,by,i in get_batch(X_train,Y_train,Y_train.shape[0],25):
            if i % 10 == 0:
                train_acc  = sess.run(accuracy,feed_dict={x:bx,y:by,keep_prob:1.0})
                print 'step %d-%d train_acc : %g' % (epoch,i,train_acc)
            summary,_  = sess.run([merged,train_step],feed_dict={x:bx,y:by,keep_prob:0.75})
    writer.add_summary(summary,epoch)

step 0-0 train_acc : 0.6
step 0-10 train_acc : 0.6
step 0-20 train_acc : 0.6
step 0-30 train_acc : 0.6
step 0-40 train_acc : 0.6
step 0-50 train_acc : 0.6
step 0-60 train_acc : 0.6
step 0-70 train_acc : 0.6
step 0-80 train_acc : 1
step 1-0 train_acc : 0.6
step 1-10 train_acc : 0.6
step 1-20 train_acc : 0.6
step 1-30 train_acc : 0.6
step 1-40 train_acc : 0.6
step 1-50 train_acc : 0.6
step 1-60 train_acc : 0.6
step 1-70 train_acc : 0.6
step 1-80 train_acc : 1
step 2-0 train_acc : 0.6
step 2-10 train_acc : 0.6
step 2-20 train_acc : 0.6
step 2-30 train_acc : 0.6
step 2-40 train_acc : 0.6
step 2-50 train_acc : 0.6
step 2-60 train_acc : 0.6
step 2-70 train_acc : 0.6
step 2-80 train_acc : 1
step 3-0 train_acc : 0.6
step 3-10 train_acc : 0.6
step 3-20 train_acc : 0.6
step 3-30 train_acc : 0.6
step 3-40 train_acc : 0.6
step 3-50 train_acc : 0.6
step 3-60 train_acc : 0.6
step 3-70 train_acc : 0.6
step 3-80 train_acc : 1
step 4-0 train_acc : 0.6
step 4-10 train_acc : 0.6
step 4-20 train_acc : 0.6

RuntimeError: Attempted to use a closed Session.

In [ ]:
def PrintReport(real,predict,length):
    TP = 0 
    TN = 0
    FP = 0
    FN = 0
    for i in range(length):
        TP += 1 if((real[i] == 1) and (real[i] == predict[i])) else 0
        TN += 1 if((real[i] == 0) and (real[i] == predict[i])) else 0
        FP  += 1 if((predict[i] == 1) and (real[i] != predict[i])) else 0
        FN += 1 if((predict[i] == 0) and (real[i] !=predict[i])) else 0
    precision_P = (TP*1.0/(TP+FP)) if TP+FP != 0 else 0
    precision_N = TN*1.0/(TN+FN) if TN+FN !=0 else 0
    recall_P = TP*1.0/(TP+FN) if TP+FN !=0 else 0
    recall_N = TN*1.0/(TN+FP) if TN+FP !=0 else 0
    f1_P = (2.0/(1.0/precision_P+1.0/recall_P)) if precision_P!=0 and recall_P!=0 else 0
    f1_N = 2.0/(1.0/precision_N+1.0/recall_N) if precision_N !=0 and recall_N !=0 else 0
    header = 'Label precision recall f1-score support'.split()
    row1 = ['0',precision_N,recall_N,f1_N,TN+FP]
    row2= ['1',precision_P,recall_P,f1_P,TP+FN]
    row3=['avg/total',0,0,0,row1[4]+row2[4]]
    for i in range(1,4):
        row3[i]=(row1[i]+row2[i])/2.0
    pt = PrettyTable()
    pt._set_field_names(header)
    pt.add_row(row1)
    pt.add_row(row2)
    pt.add_row(row3)
    acc = (TP+TN)*1.0/(length)
    print 'Test Accuracy '+str(acc)
    print str(pt)+'\n'

In [ ]:
prediction = tf.argmax(y_conv,1)
Y_now = tf.argmax(Y_test,1)
result = np.zeros(X_test.shape[0]).astype(int)
for i in range(X_test.shape[0]//25+1):
    tmp = sess.run(prediction,feed_dict = {x:X_test[i*25:min(i*25+25,X_test.shape[0])],keep_prob:1.0})
    result[i*25:min(i*25+25,X_test.shape[0])] = (np.array(tmp).astype(int))
PrintReport(Y_now.eval(),result,Y_test.shape[0])


RuntimeError: Attempted to use a closed Session.

In [ ]:
result= []
for i in range(X_train.shape[0]//25+1):
    tmp = sess.run(prediction,feed_dict = {x:X_train[i*25:min(i*25+25,X_train.shape[0])],keep_prob:1.0})
    result[i*25:min(i*25+25,X_train.shape[0])] = (np.array(tmp).astype(int))
Y_now = tf.argmax(Y_train,1)
PrintReport(Y_now.eval(),result,Y_train.shape[0])